In [16]:
import pandas as pd
import numpy as np
import os
import subprocess

In [17]:
#load all tsvs from /opt/notebooks/freesurfer_stats
dfs_dict={}
for file in os.listdir('/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/mris/'):
    if file.endswith('.csv'):
        if file.startswith("lh"):
            lh_df = pd.read_csv('/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/mris/'+file)
            rh_df = pd.read_csv('/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/mris/rh'+file[2:])
            lh_df.rename(columns={lh_df.columns[0]: 'ID'}, inplace=True)
            rh_df.rename(columns={rh_df.columns[0]: 'ID'}, inplace=True)
            df = pd.merge(lh_df, rh_df, on='ID')
            df["ID"] = df["ID"].str.replace("sub-", "",)
            name = file[3:].replace(".csv", "")
        elif file.startswith("rh") == False:
            df = pd.read_csv('/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/mris/'+file)
            df.rename(columns={df.columns[0]: 'ID'}, inplace=True)
            df["ID"] = df["ID"].str.replace("sub-", "")
            name = file.replace(".csv", "")
        else:
            continue
        dfs_dict[name] = df


In [18]:
for key in dfs_dict.keys():
    print(key)

aparc.thicknessstd
aparc.thickness
aparc.volume
aseg.volume


In [19]:
keys = ['aseg.volume', 'aparc.volume', 'aparc.thickness']
for key in list(dfs_dict.keys()):  
    if key not in keys: 
        del dfs_dict[key]

In [20]:
for key, df in dfs_dict.items():
    print(key)
    for col in df.columns:
        print(df[col].describe())

aparc.thickness
count      30256
unique     30256
top       130928
freq           1
Name: ID, dtype: object
count    30256.000000
mean         2.514051
std          0.138625
min          1.523000
25%          2.421750
50%          2.512000
75%          2.605000
max          3.166000
Name: lh_bankssts_thickness, dtype: float64
count    30256.000000
mean         2.439380
std          0.181122
min          1.084000
25%          2.316000
50%          2.432000
75%          2.555000
max          3.297000
Name: lh_caudalanteriorcingulate_thickness, dtype: float64
count    30256.000000
mean         2.492179
std          0.119263
min          1.923000
25%          2.413000
50%          2.490000
75%          2.571000
max          3.189000
Name: lh_caudalmiddlefrontal_thickness, dtype: float64
count    30256.000000
mean         1.870410
std          0.127683
min          1.175000
25%          1.782000
50%          1.867000
75%          1.954000
max          2.509000
Name: lh_cuneus_thickness, dty

In [21]:
drop_cols_aseg_vol = [
    "BrainSegVolNotVent",
    "BrainSegVolNotVentSurf",
    "eTIV",
    "MeanThickness",
    "EstimatedTotalIntraCranialVol",
    "MaskVol",
    "BrainSegVol-to-eTIV",
    "MaskVol-to-eTIV",
    "SurfaceHoles",
] 

In [22]:
for key, df in dfs_dict.items():
    drop_cols = [col for col in df.columns if any(drop_col in col for drop_col in drop_cols_aseg_vol)]
    df.drop(columns=drop_cols, inplace=True)
    dfs_dict[key] = df
    print(len(df))
    print(len(df.columns))

30256
69
30256
69
30256
57


In [23]:
for mri in keys:
    print(mri)
    df = pd.read_csv('/zi/home/esra.lenz/Documents/00_HITKIP/00_CLIP/00_NAKO/02_Validation_of_UKB/00_data/freesurfer_finished/'+mri+'.csv')
    print(len(df.columns))

aseg.volume
57
aparc.volume
69
aparc.thickness
69


In [24]:
# Create the directory if it does not exist
output_dir = '../00_data/freesurfer_finished/'
os.makedirs(output_dir, exist_ok=True)

# Save individual dataframes
for key, df in dfs_dict.items():
    print(len(df.columns))
    df.to_csv(os.path.join(output_dir, f'{key}.csv'), index=False)


69
69
57


In [25]:
#command = "dx upload -r /opt/notebooks/freesurfer_finished/*.csv --path project-GvJ4qQ8Jk6k7QFZ0Jzjp7vJ6:/Users/Esra_Lenz/02_Data/00_mri_tables/00_important_freesurfer_stats/"
#subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)